In [1]:
import pandas as pd
from Testmodule import 資產負債表, 綜合損益表, 證交所產業分類
from datetime import date

# 年度
current_date = date.today()
user_year = current_date.year

# 上一季資料
current_quarter = (current_date.month - 1) // 3 + 1
user_quarter = current_quarter - 1 
if user_quarter == 1:
    user_quarter = 4
    user_year -= 1  #如果是第一季，user_year改為上個年度

# user_year = int(input('請輸入西元年份'))
# user_quarter = int(input('請輸入上個季度'))

income_statement = 綜合損益表.綜合損益表by季(user_year, user_quarter)
balance_sheet = 資產負債表.資產負債表by季(user_year, user_quarter)

#兩張table都有獨立的key，利用這兩個key值將兩張table結合
resultdf = income_statement.merge(balance_sheet,on='Key',how='inner')
resultdf = resultdf.drop(columns=['公司 代號_y', '年度_y', '季別_y', '公司名稱_y']) #刪掉重複欄位
resultdf.rename(columns = {'公司 代號_x':'公司代號', '年度_x':'年度', '季別_x':'季別', '公司名稱_x':'公司名稱' }, inplace = True) #重新命名欄位
resultdf['公司代號'] = resultdf['公司代號'].astype(str) #變更欄位的type，用於merge用
category = 證交所產業分類.industry()
resultdf = resultdf.merge(category,how = "inner",on = "公司代號")
resultdf



,公司代號,年度,季別,公司名稱,營業收入,營業成本,原始認列生物資產及農產品之利益（損失）,生物資產當期公允價值減出售成本之變動利益（損失）,營業毛利（毛損）,未實現銷貨（損）益,...,歸屬於母公司業主之權益合計,共同控制下前手權益,合併前非屬共同控制股權,非控制權益,權益總計,待註銷股本股數（單位：股）,預收股款（權益項下）之約當發行股數（單位：股）,母公司暨子公司所持有之母公司庫藏股股數（單位：股）,每股參考淨值,產業別
0,1101,112,3,台泥,80966464,66737333,--,--,14229131,--,...,222398369,--,--,22328122,244726491,--,0,20000000,30.32,水泥工業
1,1101,112,2,台泥,53964171,45509992,--,--,8454179,--,...,214284220,--,--,20435231,234719451,--,0,20000000,29.21,水泥工業
2,1101,112,1,台泥,26295929,23826708,--,--,2469221,--,...,220077058,--,--,20003976,240081034,--,0,24000000,30.02,水泥工業
3,1101,111,4,台泥,113929706,104345739,--,--,9583967,--,...,217894766,--,--,20381014,238275780,--,0,4000000,29.64,水泥工業
4,1101,111,3,台泥,79274330,74872991,--,--,4401339,--,...,201413996,--,--,20256450,221670446,--,0,4000000,29.05,水泥工業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4767,9958,112,3,世紀鋼,9539392,8275189,--,--,1264203,--,...,7895925,--,--,3434276,11330201,--,0,0,33.59,鋼鐵工業
4768,9958,112,2,世紀鋼,5894102,5299030,--,--,595072,--,...,7354916,--,--,3214123,10569039,--,0,0,31.53,鋼鐵工業
4769,9958,112,1,世紀鋼,2675284,2510328,--,--,164956,--,...,6966880,--,--,3226055,10192935,--,0,0,29.98,鋼鐵工業
4770,9958,111,4,世紀鋼,9222058,8779554,--,--,442504,--,...,6959252,--,--,3281807,10241059,--,0,0,29.95,鋼鐵工業


In [5]:
# 把resultdf另外用project這個dictionary給複製出來，避免改動到原始爬蟲資料
project = {}
project['公司代號'] = resultdf['公司代號']
project['公司名稱'] = resultdf['公司名稱']
project['產業別'] = resultdf['產業別']
project['年度'] = resultdf['年度'] 
project['季別'] = resultdf['季別']
project['基本每股盈餘_EPS'] = resultdf['基本每股盈餘（元）'] #EPS
project['營業收入'] = resultdf['營業收入']
project['權益總計'] = resultdf['權益總計'] #股東權益
project['資產總計'] = resultdf['資產總計'] #資產總額
project['淨利'] = resultdf['本期綜合損益總額'] #淨利

project = pd.DataFrame(project)
project

,公司代號,公司名稱,產業別,年度,季別,基本每股盈餘_EPS,營業收入,權益總計,資產總計,淨利
0,1101,台泥,水泥工業,112,3,0.82,80966464,244726491,472917462,10430034
1,1101,台泥,水泥工業,112,2,0.64,53964171,234719451,456341649,1142361
2,1101,台泥,水泥工業,112,1,0.20,26295929,240081034,461648368,2506413
3,1101,台泥,水泥工業,111,4,0.74,113929706,238275780,460808961,5302008
4,1101,台泥,水泥工業,111,3,0.38,79274330,221670446,458733599,1910412
...,...,...,...,...,...,...,...,...,...,...
4767,9958,世紀鋼,鋼鐵工業,112,3,2.52,9539392,11330201,34735722,656662
4768,9958,世紀鋼,鋼鐵工業,112,2,1.20,5894102,10569039,33949249,212730
4769,9958,世紀鋼,鋼鐵工業,112,1,0.20,2675284,10192935,33597183,-8900
4770,9958,世紀鋼,鋼鐵工業,111,4,1.59,9222058,10241059,29360142,70820


In [39]:
# dataset1
import pandas as pd
from pandas.core.reshape.pivot import pivot
# 先將project的年度、季別合併，變成date的形式後排序，之後繪圖座標軸才會正確
project['Date'] = (project['年度'] + 1911).astype(str)+ (project['季別']*3-2).astype(str)
project['Date'] = pd.to_datetime(project['Date'], format='%Y%m')

import datetime
p_time = project #複製一個要修改的p_time避免動到原本project
p_time = p_time.drop_duplicates(subset=['Date']) #把重複的時間刪掉，因此只會有5個時間從最新(當季)到最舊(前5季)
time_list = []
for i in range(5):
    time_list.append(p_time.iat[i,-1].strftime("%Y-%m-%d")) #用for迴圈把每個時間加到time_list
time_list = sorted(time_list, reverse=True) #由最新的季別排到最早的季別

# exclude1: 每年營收呈現負成長
p1 = project[['公司名稱','Date','營業收入']]
p1.sort_values(by = ['公司名稱','Date']) #排序資料
p1 = p1.pivot(index = '公司名稱',columns = 'Date',values = '營業收入') #轉置表格 #n=958
p1 = p1.dropna(subset = time_list, how = 'any') #刪除任一季的營業額為missing,n=943
for i in range(len(time_list)-1):
    month_current = time_list[i].split('-')[1]
    month_previous = time_list[i+1].split('-')[1]
    if month_current != '01': #大部份第一季一定比前一年的第四季少營收，因此不比較
        p1 = p1[~(p1[time_list[i]] < p1[time_list[i+1]])] #n=942 用time_list取代固定季別
#p1 = p1[~((p1['2023-07-01'] < p1['2023-04-01']) | (p1['2023-04-01'] < p1['2023-01-01']) | (p1['2022-10-01'] < p1['2022-07-01']))] #n=942 

# exclude2: 任一季EPS為負數
p2 = project[['公司名稱','Date','基本每股盈餘_EPS']]
p2.sort_values(by = ['公司名稱','Date'])
p2 = p2.pivot(index = '公司名稱',columns = 'Date',values = '基本每股盈餘_EPS') #n=958
p2 = p2.dropna(subset = time_list, how = 'any') #刪除任一季的基本每股盈餘_EPS為missing,n=943
for time_ymd in time_list:
    p2 = p2[~(p2[time_ymd] < 0)] #n=619 用迴圈跑time_list取代固定季別，之後可以擴充更多季別
#p2 = p2[~((p2['2023-07-01'] < 0) | (p2['2023-04-01'] < 0) | (p2['2023-01-01'] < 0) | (p2['2022-10-01'] < 0) | (p2['2022-07-01'] < 0))] #n=620

# exclude1+exclude2 #n=549
# p1 = p1[['公司名稱']]
# p2 = p2[['公司名稱']]
# print(pd.merge(p1, p2, on='公司名稱', how='inner'))

# exclude3: 任一指標低於該產業平均

Date,2022-07-01,2022-10-01,2023-01-01,2023-04-01,2023-07-01
公司名稱,,,,,
ABC-KY,269566.0,390302.0,82709.0,161187.0,266940.0
AES-KY,11625088.0,15052408.0,2920778.0,5291073.0,7545131.0
GIS-KY,92654570.0,125461361.0,14374739.0,35036246.0,52426194.0
IKKA-KY,2696711.0,3618633.0,758933.0,1603159.0,2595594.0
TPK-KY,75378508.0,97180969.0,17324195.0,33552820.0,51606110.0
...,...,...,...,...,...
鼎固-KY,7084815.0,9221577.0,3618088.0,4268446.0,7441934.0
鼎基,1997828.0,2607212.0,617346.0,1272045.0,2001150.0
鼎炫-KY,1857587.0,2309212.0,387333.0,899731.0,1379727.0


In [14]:
import pandas as pd
import matplotlib.pyplot as plt

# 先將project的年度、季別合併，變成date的形式後排序，之後繪圖座標軸才會正確
project['Date'] = (project['年度'] + 1911).astype(str)+ (project['季別']*3-2).astype(str)
project['Date'] = pd.to_datetime(project['Date'], format='%Y%m')

project = project.sort_values(by='Date')
project = pd.DataFrame(project)


def plot_company_data(company_code):

    company_data = project[project['公司代號'] == company_code]
    plt.figure(figsize=(10, 6))

    # Define a custom date formatter
    date_format = "%YQ%q"
    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter(date_format))

    # Plot the data
    plt.plot(company_data['Date'], company_data['營業收入'], label='營業收入', marker='o')
    plt.title('Revenue')
    
    # 設定 x 軸刻度為 Quarter，並使用自定義的日期格式
    quarters = company_data['Date'].dt.to_period("Q")
    quarters_str = quarters.astype(str)
    plt.xticks(company_data['Date'].unique(), quarters_str.unique())
    
    # 設定 y 軸刻度平均分成 5 等份
    min_value = company_data['營業收入'].min()
    max_value = company_data['營業收入'].max()
    tick_interval = (max_value - min_value) / 5
    plt.yticks([min_value + i * tick_interval for i in range(6)])
    
    plt.xlabel('Quarter')
    plt.ylabel('Amount')
    plt.legend()
    plt.grid(True)
    plt.show()

company_code = int(input('輸入公司代號'))
plot_company_data(company_code)
# test

ValueError: cannot convert float NaN to integer

Error in callback <function _draw_all_if_interactive at 0x000001B7EAB8C0E0> (for post_execute):


ValueError: cannot convert float NaN to integer

ValueError: cannot convert float NaN to integer

<Figure size 1000x600 with 1 Axes>